````{=typst}
#import "../common/template.typ": conf
#import "../common/dotenv.typ": env

#show: doc => conf(
  title: [第五章作业报告],
  authors: (
    (
      name: env.STUDENT_NAME,
      studentID: env.STUDENT_ID,
      email: env.STUDENT_EMAIL,
    ),
  ),
  doc,
)

= 习题一

== 题目

编写函数 filter_chars()，删除字符串中的非英文字母。

== 思路

正则提取字母，上次作业中已经实现，同时具备保序性。

== 测试

```python
# test_chapter5.py

from chapter5.chapter5_1 import filter_chars


def test_filter_chars():
  assert filter_chars('a1b2c3') == 'abc'
  assert filter_chars('a1b2c3!@#') == 'abc'
  assert filter_chars('a1b2c3!@#🤣🤣') == 'abc'
  assert filter_chars('你好qaq') == 'qaq'
  pass
```

所有测试均通过。

== 代码

````


In [ ]:
from re import findall


def filter_chars(string: str) -> str:
  return ''.join(findall(r'[a-zA-Z]', string))


def main() -> None:
  raw = input()
  print('input: ', raw)
  print(filter_chars(raw))
  return


main()

````{=typst}
= 习题二

== 题目

接受用户输入的四则运算表达式，使用栈进行表达式求值。算术表达式是形如 “1 - 2 + 3 \* 4” 这样的表达式。为简单计，不考虑括号。

== 思路

原则上等价于 `return eval(f'{a} {op} {b}')`。

考虑到顺序问题，使用两个栈，一个存储操作数，一个存储操作符。遍历表达式，遇到操作数则入栈，遇到操作符则与栈顶元素比较优先级，如果优先级高则入栈，否则弹出栈顶元素进行运算。提供了基于 `split` 的简单实现和基于字符串遍历的实现。

== 测试

```python
# test_chapter5.py

from chapter5.chapter5_2 import mock, calc, calc_2


def test_mock():
  assert mock(1, 2, '+') == 3
  assert mock(1, 2, '-') == -1
  assert mock(1, 2, '*') == 2
  assert mock(1, 2, '/') == 0.5
  pass


def test_calc():
  assert calc('1 + 2') == 3
  assert calc('1 + 2 * 3') == 7
  assert calc('1 + 2 * 3 - 4') == 3
  pass


def test_calc_2():
  assert calc_2('1 + 2') == 3
  assert calc_2('1 + 2 * 3') == 7
  assert calc_2('1+2*3-4') == 3
  pass
```

所有测试均通过。

== 代码

````


In [ ]:
def mock(a: int | float | None, b: int | float | None, op: str) -> int | float | None:
  """模拟运算"""
  if a is None or b is None:
    return None

  if op == '+':
    return a + b
  if op == '-':
    return a - b
  if op == '*':
    return a * b
  if op == '/':
    if b == 0:
      return None
    return a / b

  return None


def calc(raw: str) -> int | float:
  nums = []
  ops = []

  priority = {'+': 1, '-': 1, '*': 2, '/': 2}

  for token in raw.split():
    if token.isdigit():
      nums.append(int(token))
      pass
    else:
      while ops and priority[ops[-1]] >= priority[token]:
        nums.append(mock(nums.pop(-2), nums.pop(), ops.pop()))
        pass
      ops.append(token)
      pass

    pass

  opsLen = len(ops)

  while opsLen:
    opsLen -= 1
    nums.append(mock(nums.pop(-2), nums.pop(), ops.pop()))
    pass

  return nums[0]


def calc_2(raw: str) -> int | float:
  nums = []
  ops = []

  priority = {'+': 1, '-': 1, '*': 2, '/': 2}

  temp = 0

  for char in raw:
    if char.isdigit():
      temp = temp * 10 + int(char)
      pass
    elif char in priority:
      nums.append(temp)
      temp = 0
      while ops and priority[ops[-1]] >= priority[char]:
        nums.append(mock(nums.pop(-2), nums.pop(), ops.pop()))
        pass
      ops.append(char)
      pass
    pass

  nums.append(temp)

  opsLen = len(ops)

  while opsLen:
    opsLen -= 1
    nums.append(mock(nums.pop(-2), nums.pop(), ops.pop()))
    pass

  return nums[0]


def main():
  raw = input()
  print('input: ', raw)
  print(calc(raw))
  return


main()

````{=typst}
= 习题三

== 题目

编写程序，随机生成 4 个 1~10 之间的整数，允许重复，判断其能否算出 24。计算规则为：（1）允许使用加减乘除等运算；（2）允许使用括号改变优先级。

== 思路

暴力搜索，遍历所有可能的运算符排列组合，计算结果，判断是否为 24。考虑到中缀表达式符号位置固定，使得在使用括号的情况下同一结构的表达式可以对应不同的结果，因此使用没有歧义的后缀表达式（逆波兰表达式）进行计算，所有组合均用后缀表达式生成。

== 测试

```python
# test_chapter5.py

from chapter5.chapter5_3 import calcRPN, gen, genRPNs, judge24

def test_gen():
  assert len(gen()) == 4

  print()  # print a blank line here
  print(*gen())

  with open('__tests__/__snapshots__/test_chapter5_3_gen.snap', 'w') as f:
    for _ in range(10):
      print(*gen(), file=f)
      pass
    pass
  pass


def test_calcRPN():
  assert calcRPN([1, 2, '+', 3, '*']) == 9
  assert calcRPN([1, 2, '+', 3, '*', 4, '-']) == 5
  pass


def test_genRPNs():
  assert len(genRPNs([1, 2, 3, 4])) == 320

  with open('__tests__/__snapshots__/test_chapter5_3_genRPNs.snap', 'w') as f:
    for item in genRPNs([1, 2, 3, 4]):
      print(item, file=f)
      pass
    pass
  pass


def test_judge24():
  assert judge24([1, 2, 3, 7])
  assert judge24([1, 2, 3, 4])
  assert judge24([1, 2, 3, 5])

  assert not judge24([1, 1, 1, 1])
  assert not judge24([1, 1, 1, 2])

  with open('__tests__/__snapshots__/test_chapter5_3_judge24.snap', 'w') as f:
    for _ in range(10):
      nums = gen()
      print(nums, judge24(nums), file=f)
      pass
    pass
  pass
```

所有测试均通过，生成的快照文件如下：

- `__tests__/__snapshots__/test_chapter5_3_gen.snap`

```txt
4 1 1 7
11 7 11 9
10 8 4 7
7 10 11 11
2 3 7 4
7 2 8 11
9 1 5 9
10 9 11 8
7 3 7 7
6 3 2 10
```

- `__tests__/__snapshots__/test_chapter5_3_genRPNs.snap`

```txt
[1, 2, 3, 4, '+', '+', '+']
[1, 2, 3, '+', 4, '+', '+']
[1, 2, 3, '+', '+', 4, '+']
[1, 2, '+', 3, 4, '+', '+']
[1, 2, '+', 3, '+', 4, '+']
[1, 2, 3, 4, '+', '+', '-']
[1, 2, 3, '+', 4, '+', '-']
[1, 2, 3, '+', '+', 4, '-']
[1, 2, '+', 3, 4, '+', '-']
[1, 2, '+', 3, '+', 4, '-']
...省略 310 行...
```

- `__tests__/__snapshots__/test_chapter5_3_judge24.snap`

```txt
[4, 8, 9, 2] False
[6, 10, 3, 4] (True, [6, 10, '*', 3, '/', 4, '+'])
[6, 9, 9, 1] (True, [6, 9, 9, '+', '+', 1, '*'])
[1, 9, 1, 6] False
[11, 2, 8, 3] (True, [11, 2, 8, 3, '+', '+', '+'])
[10, 8, 11, 3] False
[9, 5, 9, 1] (True, [9, 5, 9, 1, '+', '+', '+'])
[9, 6, 5, 2] (True, [9, 6, 5, '*', 2, '/', '+'])
[4, 6, 3, 11] (True, [4, 6, 3, 11, '+', '+', '+'])
[6, 6, 5, 7] (True, [6, 6, 5, 7, '+', '+', '+'])
```

== 代码

````


In [ ]:
from random import randint
from itertools import product
from copy import copy
from typing import List

# 复用第二题的 mock 函数
# from .chapter5_2 import mock


def gen(n=4) -> List[int]:
  """生成器"""
  return [randint(1, 11) for _ in range(n)]


def calcRPN(tks) -> int | float | None:
  """逆波兰表达式求值"""
  lg = len(tks)
  stk = copy(tks)
  tmp = 2
  while lg > 1:
    if stk[tmp] in ('+', '-', '*', '/'):
      stk[tmp] = mock(stk[tmp - 2], stk[tmp - 1], stk[tmp])
      stk.pop(tmp - 1)
      stk.pop(tmp - 2)
      tmp -= 1
      lg -= 2
      pass
    else:
      tmp += 1
    pass
  return stk[0]


def genRPNs(nums):
  """生成逆波兰表达式"""
  ops = list(product(['+', '-', '*', '/'], repeat=3))

  res = []

  def mergeList(lt1, lt2):
    """
    考虑到列表大小较小，这里直接穷举所有逆波兰表达式的符号排列可能

    以 lt1 = [a, b, c, d], lt2 = [+, -, *] 为例，共五种结合顺序：
      - [a, b, c, d, +, -, *] —— ((ab)c)d
      - [a, b, c, +, d, -, *] —— (a(bc))d
      - [a, b, c, +, -, d, *] —— a((bc)d)
      - [a, b, +, c, d, -, *] —— (ab)(cd)
      - [a, b, +, c, -, d, *] —— a(b(cd))
    """
    return [
      lt1 + lt2,
      lt1[:3] + lt2[0:1] + lt1[3:] + lt2[1:],
      lt1[:3] + lt2[0:2] + lt1[3:] + lt2[2:],
      lt1[:2] + lt2[0:1] + lt1[2:] + lt2[1:],
      lt1[:2] + lt2[0:1] + lt1[2:3] + lt2[1:2] + lt1[3:] + lt2[2:],
    ]

  for op in ops:
    res.extend(mergeList(nums, list(op)))
  return res


def judge24(nums):
  """判断是否能组合成 24"""
  for item in genRPNs(nums):
    tmp = calcRPN(item)
    if tmp is not None and abs(tmp - 24) < 1e-6:
      return (True, item)
  return False


def main() -> None:
  """主函数"""
  for _ in range(10):
    nums = gen()
    print(nums, judge24(nums))
    pass
  return


main()

````{=typst}
= 习题四

== 题目

编写程序，随机生成 13 张扑克牌（无大小王牌），统计其牌点（point）。扑克牌牌张（card）有牌花（suit）与牌阶 （rank）两种属性。使用字符串 “23456789TJQKA” 表示牌阶，每一牌阶用单个字符表示，顺序从小到大，其中 “T” 表示 10。使用字符串 “CDHS” 表示牌花，每种牌花用单个字符表示，顺序从小到大，分别为梅花（club）、方块（diamond）、红桃（heart）和黑桃（spade)。牌点规定如下：“A” 记为 4 点，“K” 记为 3 点，“Q” 记为 2 点，“J” 记为 1 点，其他牌记为 0 点。如果熟悉桥牌（bridge），编写函数，返回其合理开叫（opening bidding）。叫牌时使用 “p” 表示通过，“NT” 表示无将（no trump），无将大于全部花色。叫牌时，可能需要根据叫牌或定约（contract）情况修正牌点。例如，无将定约时应增加长套牌型点，扣除单张大牌点；有将定约时亦存在类似规则。

== 思路

OOP 设计，定义 `Cards` 类：定义牌组，初始化时洗牌并发牌，然后计算牌点。根据题目要求，此处默认只计算第一组牌的牌点。

由于我不熟悉桥牌，此处没有给出根据桥牌规则的具体实现，即并没有实现返回合理开叫的函数。

== 测试

```python
# test_chapter5.py

from chapter5.chapter5_4 import Cards, divide

def test_divide():
  assert divide([1, 2, 3, 4, 5, 6, 7, 8], 4) == [[1, 2], [3, 4], [5, 6], [7, 8]]
  assert divide([1, 2, 3, 4, 5, 6, 7, 8, 9], 3) == [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
  assert divide([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 3) == [[1, 2, 3, 4], [5, 6, 7], [8, 9, 10]]
  assert divide([1, 2, 3, 4, 5, 6, 7, 8], 3) == [[1, 2, 3], [4, 5, 6], [7, 8]]

  assert divide([(1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8)], 4) == [
    [(1, 1), (2, 2)],
    [(3, 3), (4, 4)],
    [(5, 5), (6, 6)],
    [(7, 7), (8, 8)],
  ]
  pass


def test_Cards():
  cards = Cards()

  assert cards.length == 13

  print()  # print a blank line here
  print('Hand:', cards.hand)
  print('Total Point:', cards())

  with open('__tests__/__snapshots__/test_chapter5_4_cards.snap', 'w') as f:
    print('Hand:', cards.hand, file=f)
    print('Total Point:', cards(), file=f)
    pass

  pass
```

所有测试均通过，生成的快照文件如下：

- `__tests__/__snapshots__/test_chapter5_4_cards.snap`

```txt
Hand: [('Q', 'H'), ('Q', 'D'), ('4', 'H'), ('6', 'D'), ('2', 'S'), ('5', 'S'), ('T', 'C'), ('9', 'C'), ('T', 'H'), ('3', 'S'), ('9', 'D'), ('2', 'C'), ('5', 'H')]
Total Point: 4
```

== 代码

````


In [ ]:
from random import shuffle
from typing import List


def divide(lt: List, n: int) -> List[List]:
  """将一个数组尽量均分成 n 个子数组，溢出的部分将分配到前面的子数组中"""
  k, m = divmod(len(lt), n)
  return [lt[i * k + min(i, m) : (i + 1) * k + min(i + 1, m)] for i in range(n)]


class Cards:
  # 定义一副牌及所有牌的功能
  CARD_RANK = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
  CARD_SUIT = ['C', 'D', 'H', 'S']
  CARD_POINT = {'A': 4, 'K': 3, 'Q': 2, 'J': 1}
  BRIDGE_STATUS = ['NT', 'p']  # 由于我不熟悉桥牌，此处没有给出根据桥牌规则的具体实现

  def __init__(self, n=4) -> None:
    # 定义牌组
    self.CARD_DECK = [(rank, suit) for rank in self.CARD_RANK for suit in self.CARD_SUIT]

    # 洗牌
    shuffle(self.CARD_DECK)

    # 发牌
    self.hands = divide(self.CARD_DECK, n)

    # 这里默认取第一组作为手牌
    self.hand = self.hands[0]
    self.length = len(self.hand)

    self.point = 0
    self.table = None
    return

  def getHand(self):
    """此处手牌会以元组的形式返回，元组的第一个元素是牌面，第二个元素是花色"""
    return self.hand

  def getPoint(self, card: str) -> int:
    """根据牌面计算一张牌的牌点"""
    return self.CARD_POINT.get(card, 0)

  def getTotalPoint(self):
    """计算手牌的总牌点"""
    self.point = sum([self.getPoint(card[0]) for card in self.hand])
    return self.point

  def getAllTotalPoint(self):
    """返回所有牌组的各自总牌点"""
    self.table = [sum([self.getPoint(card[0]) for card in hand]) for hand in self.hands]
    return self.table

  def __call__(self):
    """便于调用，计算手牌的总牌点"""
    return self.getTotalPoint()

  pass


def main() -> None:
  cards = Cards()

  print('Hand:', cards.hand)
  print('Total Point:', cards())
  return


main()

```{=typst}
= 结论

对于一些随机测试题，快照结果可能与记录不一致，您可以自行运行测试代码进行验证。整体来说，本次作业完成得还好。

由于打包前后项目结构有所调整，测试文件需要经过调整才能正常运行，去除 `import` 语句的包头名称即可，您可以在 #link(env.TARGET_URL)[此处] 找到完整的项目结构。
```
